# GPT2 Zero-Shot CommonsenseQA Experiment
**Last Edited On: 5/30/2023**<br>
**Last Edited By: Kyle Williams**

**Motivation:** This file contains a scrpt for finding the zero-shot performance of GPT2 on CommonsenseQA. Our performance metric is accuracy, where the model's generated answer will be considered correct if it contains the correct multiple-choice answer. Because we don't have the test set labels, we will be reporting the accuracy on the development set instead. 

**Resources:**
- [A GPT2 Fine-tuning tutorial](https://colab.research.google.com/drive/1QIMbIbkDo7TAiNB2xoI5L53dnmNbYV6h#scrollTo=NKGBoVwuhM4H)
- [Fine-tuning GPT2 tricks](https://github.com/falloutdurham/beginners-pytorch-deep-learning/blob/master/chapter9/Chapter9.5.ipynb)

In [25]:
'''
Necessary Imports

TODO: I added all imports from the resource above, but we aren't fine-tuning in this notebook, so some can be deleted
'''
import pickle
import torch 
import os

from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DisjunctiveConstraint

In [26]:
'''
Load GPT2 Model and Corresponding Tokenizer. 
Then save them to the device for GPU computation. 

TODO: These should be the same tokenizer used to create the files in /data/tensor_splits. For now, I'm just 
      copy-pasting the code, but there's gotta be a less error-prone way to make changes if needed. 
'''
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'
model = GPT2LMHeadModel.from_pretrained('gpt2-medium',
                                        pad_token_id=tokenizer.eos_token_id)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
print(device)

cpu


In [35]:
'''
Define Dataset Object, then create a Dataloader Object to Feed Prompts into the Model. 
'''
# Dataloader creation parameters. Must be here because keywords are loaded outside of the
# CommonsenseDataset object, but these values must be consistent
train_split = False
constrained = True 

# Define paths to files based on parameters above
cwd = os.getcwd()
parent_path = '/'.join(cwd.split('/')[0:-1]) # removes the innermost folder (currently /experiments)
prompt_file = 'TRAINsplit_prompts.pkl' if train_split else 'DEVsplit_prompts.pkl'
answer_file = 'TRAINsplit_answers.pkl' if train_split else 'DEVsplit_answers.pkl'
keywords_file = 'TRAINsplit_keywords.pkl' if train_split else 'DEVsplit_keywords.pkl'

class CommonsenseDataset(Dataset):
    def __init__(self, train_split=False, constrained=True):
        # Load prompt and answer dumps for the inputted split
        prompts = None
        with open(parent_path + f'/data/prompt_splits/{prompt_file}', 'rb') as file:
            prompts = pickle.load(file)
        with open(parent_path + f'/data/prompt_splits/{answer_file}', 'rb') as file:
            self.answers = pickle.load(file)
        if not prompts or not self.answers:
            raise IOError("Could not read one of the necessary pickle files!")

        # Tokenize input prompts and retrieve attention_masks list
        outputs = tokenizer(prompts, padding='longest', truncation=True, return_tensors='pt')
        self.input_ids = outputs['input_ids']
        self.attn_masks = outputs['attention_mask']
        
    def __len__(self):
        return len(self.answers)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx], self.answers[idx]

# Create DataLoader for CommonsenseDataset
dev_loader = DataLoader(CommonsenseDataset(train_split=train_split, constrained=constrained), 
                        batch_size=1, shuffle=False)

# Now load keywords, if desired
keywords = None
if constrained:
    with open(parent_path + f'/data/prompt_splits/{keywords_file}', 'rb') as file:
        keywords = pickle.load(file) 
        

In [36]:
keywords[0]

['bank',
 'library',
 'department store',
 'mall',
 'new york',
 'complete job',
 'learn from each other',
 'kill animals',
 'wear hats',
 'talk to each other',
 'doctor',
 'bookstore',
 'market',
 'train station',
 'mortuary',
 'fast food restaurant',
 'pizza',
 'ground up dead cows',
 'mouth',
 'cow carcus',
 'midwest',
 'countryside',
 'estate',
 'farming areas',
 'illinois',
 'own home',
 'north carolina',
 'great britain',
 'hutch',
 'outdoors',
 "mildred's coffee shop",
 'mexico',
 'diner',
 'kitchen',
 'canteen',
 'feel pleasure',
 'procreate',
 'pass water',
 'listen to each other',
 'sing',
 'literacy',
 'knowing how to read',
 'money',
 'buying',
 'money bank',
 'cry',
 'hear sounds',
 'singing',
 'arthritis',
 'making music',
 'pants',
 'record albums',
 'record store',
 'cheese',
 'wallpaper',
 'take time',
 'make noise',
 'make war',
 'make peace',
 'make haste',
 'farm house',
 'barnyard',
 'stockyard',
 'slaughter house',
 'eat cake',
 'bone',
 'charm',
 'petted',
 'lots

In [28]:
'''
GPT2 Generation Hyperparameters
'''
num_beams = 5
num_return_sequences = 1
no_repeat_ngram_size = 1
remove_invalid_values = True
do_sample = True
max_new_tokens = 10

In [24]:
'''
Feed the prompts into GPT2. Generate answers using beam search to give the most straightforward
comparison between the base model and the neurologic-enhanced one. 
'''
model_answers = [""] * len(dev_loader)
correct = 0

model.eval()
with torch.no_grad():
    for i, (input_tokens, attention_mask, answer) in enumerate(dev_loader):
        if constrained:
            constraints = DisjunctiveConstraint(keywords[i])
            input_tokens = input_tokens.to(device)
            attention_mask = attention_mask.to(device)
            answer = answer[0] # for some reason, answer is initially a tuple with one element
            
            output = model.generate(input_tokens,
                                    attention_mask=attention_mask,
                                    constraints = constraints,
                                    num_beams = num_beams,
                                    num_return_sequences = num_return_sequences,
                                    no_repeat_ngram_size = no_repeat_ngram_size,
                                    remove_invalid_values = remove_invalid_values,
                                    max_new_tokens = max_new_tokens,
                                    do_sample = do_sample)
            model_answers[i] = tokenizer.decode(output[0], skip_special_tokens=True)

            print(model_answers[i])
            
            if model_answers[i].count(answer) > 1: correct += 1 # Answer will always be there at least once (when the options are listed)

            break
        else: 
            input_tokens = input_tokens.to(device)
            attention_mask = attention_mask.to(device)
            answer = answer[0] # for some reason, answer is initially a tuple with one element
            
            output = model.generate(input_tokens,
                                    attention_mask=attention_mask,
                                    num_beams = num_beams,
                                    num_return_sequences = num_return_sequences,
                                    no_repeat_ngram_size = no_repeat_ngram_size,
                                    remove_invalid_values = remove_invalid_values,
                                    max_new_tokens = max_new_tokens,
                                    do_sample = do_sample)
            model_answers[i] = tokenizer.decode(output[0], skip_special_tokens=True)
            
            if model_answers[i].count(answer) > 1: correct += 1 # Answer will always be there at least once (when the options are listed)

print(correct / len(dev_loader))

ValueError: `nested_token_ids` has to be a list of lists, but is ['away', 'abaft', 'better themselves', 'garden', 'eastern hemisphere', 'hostile', 'clumsy', 'bookstore', 'water cooler', 'death', 'with dwight', 'hamper', 'very nice', 'speak out', 'badminton', 'keep', 'hope for peace', 'shop', 'salt', 'judge people', 'fly away', 'reason', 'hurt', 'san francisco', 'indiana', 'market', 'unbreakable', 'coffin', 'do mathematics', 'cloud over', 'graduated', 'rejection', 'energh', 'exhilliration', 'entering building', 'guide children', 'raise children', 'star can', 'sundown', 'visualize', 'tunnel', 'becoming absorbed', 'headed south', 'seattle', 'disturb', 'reputable', 'bridge', 'brass band', 'chemistry lab', 'have choice', 'perform better', 'surprised', 'spoon', 'bankruptcy', 'indian', 'banana tree', 'enabled', 'oregon', 'stew pot', 'best way', 'bank roll', 'now', 'african continent', 'nodding', 'cleanliness', 'feel lucky', 'had a party', 'unique personality', 'buy tickets', 'port', 'get tire

In [22]:
'''
Question: How often does the answer appear in the keywords provided for constrained decoding?
'''
answer_in_keywords = 0
for i, (input_tokens, attention_mask, answer) in enumerate(dev_loader):
    answer = answer[0]
    if answer in keywords[i]: 
        answer_in_keywords += 1

answer_in_keywords # With the original keywords set, this outputted 57

1221